In [1]:
import pandas as pd
from util.gene import get_gene_bnum
pd.options.display.max_columns = 100

import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.operon import get_operon_ID_set
from util.genome import NON_K12_EXP_L
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD

In [2]:
assoc_muts_df = pd.read_pickle("./data/11_df.pkl")
display(assoc_muts_df.shape, assoc_muts_df.head())

(46, 43)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source,COGs,COG links
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120014482']}
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120016992']}
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Replication, recombination and repa...","{'Replication, recombination and repair': ['EC..."
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...


In [3]:
TU_objects_df = pd.read_csv("./data/RegulonDB10/tu_objects_tmp.txt",
                            sep="\t",
                            comment='#',
                            header=None,
                            )

TU_objects_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "NUMTU",
    "TU_POSLEFT",
    "TU_POSRIGHT",
    "TU_TYPE",
    "TU_OBJECT_CLASS",
    "TU_OBJECT_ID",
    "TU_OBJECT_NAME",
    "TU_OBJECT_POSLEFT",
    "TU_OBJECT_POSRIGHT",
    "TU_OBJECT_STRAND",
    "TU_OBJECT_COLORCLASS",
    "TU_OBJECT_DESCRIPTION",
    "TU_OBJECT_SIGMA",
    "TU_OBJECT_EVIDENCE",
    "TU_OBJECT_RI_TYPE",
    "TU_OBJECT_TYPE",
    "EVIDENCE"]

In [4]:
gene_synonym_df = pd.read_csv(
    "./data/RegulonDB10/object_synonym.txt",
    sep="\t",
    comment='#',
    header=None,
    quoting=3
)
gene_synonym_df.columns = ["OBJECT_ID", "OBJECT_SYNONYM_NAME", "OS_INTERNAL_COMMENT", "KEY_ID_ORG"]
gene_synonym_df.head()

,OBJECT_ID,OBJECT_SYNONYM_NAME,OS_INTERNAL_COMMENT,KEY_ID_ORG
0,ECK120000001,EG10001,NaN,ECK12
1,ECK120000001,ECK4045,NaN,ECK12
2,ECK120000001,b4053,NaN,ECK12
3,ECK120000001,alr5,NaN,ECK12
4,ECK120000002,b0764,NaN,ECK12


In [5]:
pathway_df = pd.read_csv(
    "./data/pathways/511145.12.PATRIC.pathway.tab",
    sep="\t",
)
# Some of the entries have different entries for columsn in ec_description and ec_number and therefore can make duplicate entries when associating pathways to genes
pathway_df = pathway_df.drop(columns=['genome_id',
 'genome_name',
 'patric_id',
 'alt_locus_tag',
 'gene',
 'product',
 'ec_number',
 'ec_description',
 'pathway_id',])
pathway_df = pathway_df.drop_duplicates()

pathway_df.head()

,refseq_locus_tag,pathway_name
0,b0002,"Glycine, serine and threonine metabolism"
1,b0002,Cysteine and methionine metabolism
2,b0002,Lysine biosynthesis
6,b0003,"Glycine, serine and threonine metabolism"
7,b0004,"Glycine, serine and threonine metabolism"


In [6]:
def _get_pathways(gene_id):
    annots = []
    bnum = get_gene_bnum(gene_id, gene_synonym_df)
    df = pathway_df[pathway_df["refseq_locus_tag"]==bnum]
    for pathway in df["pathway_name"]:
        d = {"name": pathway}
        annots.append(d)
    return annots


def get_pathways(mut_row):
    annots = []
    if mut_row.exp not in NON_K12_EXP_L:
        for feat_d in mut_row["genomic features"]:
            # !!! This part only handles mutations that land within a genomic feature.
            # Need to also implement logic for handling mutations that land within intergenic regions.
            if feat_d["genetic"]:
                annots += _get_pathways(feat_d["RegulonDB ID"])
            # This will catch whether no features exist in the feat_ID_set
            # or if those features are just integenic regions.
            # Get operon genetic feature link dict through overlapping mutation and operon ranges.
            # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
            # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
            else:
                TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
                for TU_ID in TU_IDs:
                    subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]
                    for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                        if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                            annots += _get_pathways(gene_id)
    return annots


assoc_muts_df["pathways"] = assoc_muts_df.apply(get_pathways, axis=1)
assoc_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source,COGs,COG links,pathways
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120014482']},[]
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'Pyrimidine metabolism'}, {'name': '..."
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120016992']},"[{'name': 'Purine metabolism'}, {'name': 'Pyri..."
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Replication, recombination and repa...","{'Replication, recombination and repair': ['EC...",[]
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'Pyrimidine metabolism'}, {'name': '..."


In [7]:
TU_df = pd.read_csv("./data/RegulonDB10/transcription_unit.txt", sep="\t", comment='#', header=None)
TU_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "PROMOTER_ID",
    "TRANSCRIPTION_UNIT_NAME",
    "OPERON_ID",
    "TRANSCRIPTION_UNIT_NOTE",
    "TU_INTERNAL_COMMENT",
    "KEY_ID_ORG"]


operon_df = pd.read_csv("./data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [8]:
def _add_links_to_d(bnum, d, op_ID_set):
    for pathway in pathway_df[pathway_df["refseq_locus_tag"]==bnum]["pathway_name"]:
        if pathway not in d.keys(): d[pathway] = []
        d[pathway] += list(op_ID_set)


def get_pathway_operon_links_d(mut_row):
    pathway_operon_d = dict()
    if mut_row.exp not in NON_K12_EXP_L:
        for feat_d in mut_row["genomic features"]:
            if feat_d["genetic"]:
                op_ID_set = get_operon_ID_set(feat_d["RegulonDB ID"], TU_objects_df, TU_df, operon_df)
                if str(feat_d["RegulonDB ID"]) != 'nan' and feat_d["RegulonDB ID"] != '':
                    bnum = get_gene_bnum(feat_d["RegulonDB ID"], gene_synonym_df)
                    _add_links_to_d(bnum, pathway_operon_d, op_ID_set)
            else:
                TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
                for TU_ID in TU_IDs:
                    subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]

                    op_ID_set = set()
                    for _, TU_gene in subset_TU_genes_df.iterrows():
                         op_ID_set = op_ID_set | get_operon_ID_set(TU_gene["TU_OBJECT_ID"], TU_objects_df, TU_df, operon_df)

                    for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                        if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                            bnum = get_gene_bnum(gene_id, gene_synonym_df)
                            _add_links_to_d(bnum, pathway_operon_d, op_ID_set)
    return pathway_operon_d

                    
assoc_muts_df["pathway links"] = assoc_muts_df.apply(get_pathway_operon_links_d, axis=1)
assoc_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source,COGs,COG links,pathways,pathway links
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120014482']},[],{}
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D..."
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120016992']},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim..."
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Replication, recombination and repa...","{'Replication, recombination and repair': ['EC...",[],{}
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism...,{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine m

# Significance

In [9]:
# a bit behind annotations currently used in AVA MS.
# Still uses same approach to counted, just not all data-types have been consolidated into the regulators feat col


comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in assoc_muts_df.iterrows():
    for f, links in r["pathway links"].items():
        if f not in comp_mut_df.index:
            df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
            comp_mut_df = comp_mut_df.append(df)
        else:
            comp_mut_df.loc[f, "observed mutation count"] += len(links)

comp_mut_df

,observed mutation count
Pyrimidine metabolism,9
Drug metabolism - other enzymes,4
Purine metabolism,4
Starch and sucrose metabolism,1
Glycerolipid metabolism,1


In [10]:
for _, r in assoc_muts_df.iterrows():
    for feat_d in r["pathways"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [11]:
pathway_len_df = pd.read_pickle("./data/pathway_len_df.pkl")
pathway_len_df.head()

,pathway length
pathway_name,
"Glycine, serine and threonine metabolism",54804
Cysteine and methionine metabolism,36129
Lysine biosynthesis,18189
Vitamin B6 metabolism,10138
Pentose phosphate pathway,51305


In [12]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(lambda r: pathway_len_df.loc[r.index, "pathway length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df

,observed mutation count,length,proportion
Pyrimidine metabolism,9,89371,0.269703
Drug metabolism - other enzymes,4,7286,0.021988
Purine metabolism,4,154034,0.464843
Starch and sucrose metabolism,1,63933,0.192937
Glycerolipid metabolism,1,16744,0.050530


In [13]:
from collections import Counter

comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
Pyrimidine metabolism,9,89371,0.269703,451,0.0451,True
Drug metabolism - other enzymes,4,7286,0.021988,8,0.0008,True
Purine metabolism,4,154034,0.464843,9951,0.9951,False
Starch and sucrose metabolism,1,63933,0.192937,9839,0.9839,False
Glycerolipid metabolism,1,16744,0.050530,6108,0.6108,False


In [14]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
Pyrimidine metabolism,9,89371,0.269703,451,0.0451,True,0.2255,False
Drug metabolism - other enzymes,4,7286,0.021988,8,0.0008,True,0.0040,True
Purine metabolism,4,154034,0.464843,9951,0.9951,False,1.0000,False
Starch and sucrose metabolism,1,63933,0.192937,9839,0.9839,False,1.0000,False
Glycerolipid metabolism,1,16744,0.050530,6108,0.6108,False,1.0000,False


In [15]:
# # not wanting to consider mutation events with only 1 features as being significant
# comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [16]:
for i, r in assoc_muts_df.iterrows():
    for d in r["pathways"]:
        if d["name"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["name"]]["corrected significance"]
#             d["significant"] = comp_mut_df.loc[d["name"]]["final significance"]

In [17]:
assoc_muts_df.to_pickle("./data/12_df.pkl")

In [18]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["pathway links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["pathways"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 mutated pathways")
# ax.get_legend().remove()

In [19]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["pathway links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["pathways"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
            
# if len(targ_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     df = df[-20:]
#     display(df)
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#                       figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Significant mutated pathways")
#     ax.get_legend().remove()